
# Loading Libraries


In [ ]:
import numpy as np
import pandas as pd
import scipy

import statistics

import sklearn
import sklearn.ensemble
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import lime
import lime.lime_tabular

import shap


import sys

%load_ext autoreload
%autoreload 2
%matplotlib inline


import os
import missingno
import missingno as msno


from IPython.core.display import display, HTML
from tqdm.notebook import tqdm

from __future__ import print_function


# Loading Data

Lets load the dataset.
We have multiple csv files, in the dataset directory, lets load all of them in a dataframe.

In [ ]:

local_dataset_dir = '../dataset/ics_power_system/binaryAllNaturalPlusNormalVsAttacks/'

In [ ]:
import os
for dirname, _, filenames in os.walk(local_dataset_dir):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
all_files = []
for dirname, _, filenames in os.walk(local_dataset_dir):
    for filename in filenames:
        all_files.append(os.path.join(dirname, filename))

df_original = pd.concat((pd.read_csv(f) for f in all_files))

# train = df_original.sample(n =700, random_state=1) 
# test =  df_original.sample(n =300, random_state=5) 


df_original.reset_index(drop=True, inplace=True)
df_original['marker'].value_counts().values

In [ ]:
df_original.dtypes

- Lets treat the missing values. If there are `inf` values we will replace with `nan`.

- Finally we will check if there are any missing value.

- We are using https://pypi.org/project/missingno/ for analyzing and visualizing missing value. Use of this library is totally optional.

In [ ]:
df_original.replace([np.inf, -np.inf], np.nan, inplace= True)
df_original.isnull().values.any()

In [ ]:
df_original.dropna(how='any', inplace= True)
cols_numeric = df_original._get_numeric_data().columns

df_original[cols_numeric] = df_original[cols_numeric].astype('float')

df_original[cols_numeric] = (df_original[cols_numeric]- df_original[cols_numeric].mean())/df_original[cols_numeric].std()

In [ ]:
X = df_original.iloc[:, :-1]

In [ ]:
df_original

In [ ]:
# Just renaming to `label` column, also optional
df_original.rename(columns={'marker': 'label'}, inplace = True)

- For the sake of simplicity we are sampling 700 datapoint for training dataset and 300 for the testing dataset. 
- Larger dataset will result better prediction and interpretability result but will require more computation time. 

In [ ]:
X_train = df_original.iloc[:, :-1].sample(n =700, random_state=1) 
X_test = df_original.iloc[:, :-1].sample(n =300, random_state=5) 

y_train =  df_original.loc[X_train.index]['label']
y_test = df_original.loc[X_test.index]['label']

In [ ]:
y_train.replace(['Attack'], 1 , inplace= True)
y_train.replace(['Natural'], 0, inplace= True)

y_test.replace(['Attack'], 1 , inplace= True)
y_test.replace(['Natural'], 0, inplace= True)

In [ ]:
df_original.describe().T

#### Missing value

In [ ]:
df_original.isnull().sum()

In [ ]:
msno.matrix(df_original)

#### Data insight

Data contains '-' as none. So we need to remove '-' as null

In [ ]:
[y_train.value_counts().tolist(), y_test.value_counts().tolist()]

In [ ]:
import seaborn as sns

fig, ax = plt.subplots(figsize=(15,8))
size = 0.3
vals = np.array([y_train.value_counts().tolist(), y_test.value_counts().tolist()])

#cmap = plt.get_cmap("tab20")
outer_colors = plt.get_cmap("Pastel1")(np.array([1,3]))
inner_colors = plt.get_cmap("Pastel1")(np.array([0,2]*2))

b1 = ax.pie(vals.sum(axis=1), radius=1,\
            colors=outer_colors, \
            wedgeprops=dict(width=size, edgecolor='w'),\
           pctdistance=0.85,autopct='%.2f%%',)

b2 = ax.pie(x=vals.flatten(),radius=1-size,\
       colors=inner_colors, wedgeprops=dict(width=size, edgecolor='w'),\
            pctdistance=0.8, labeldistance=0.4,\
           autopct='%.2f%%',)

handles, labels = ax.get_legend_handles_labels()

#ax.legend(["Training", "Testing","Not Attack", "Attack"])
          
ax.legend(labels = ["Training", "Testing","Not Attack", "Attack"], 
           bbox_to_anchor=(0.3, 0.3, 1.0, 1.0), loc='right',
           ncol=1,  borderaxespad=2.0, prop={'size': 14})

#ax.set(title="Pie plot with `ax.bar` and polar coordinates")

# Add counts above the two bar graphs

ax.set(aspect="equal")

plt.show()

# Reward Modification

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
rf = sklearn.ensemble.RandomForestClassifier(n_estimators=10)

We are choosing random forest as prediction model. Any ML algorithm can be choosen but we are going for simplicity.

In [ ]:
model = rf.fit(X_train.values, y_train.values)
y_pred = model.predict(X_test.values)
target_names = ['Attack', 'Natural']

In [ ]:
print(classification_report(y_test, y_pred, target_names=target_names))

**With smaller dataset the result will not be very impressive, but will work for us. For better prediction result sample larger dataset.**

In [ ]:
probs = model.predict_proba(X_test)
prob1 = probs[:,1]
print(prob1)

In [ ]:
def get_coalition():
    length = len(X_test.columns)
    coalition = np.random.randint(low=0, high=2, size=length)
    return coalition

In [ ]:
print(X_test.iloc[1])

In [ ]:
import random
def get_mean_constant(coalition, instance, y_label):
    arys = []
    for j in range(100):
        l1 = []
        for i in range(len(coalition)):
            if coalition[i] == 1:
                l1.append(instance[i])
            elif coalition[i] == 0:
                rand = random.choice(X_test.iloc[:,i].values)
                l1.append(rand)
        arys.append(l1)
    if y_label == 1:
        conf = model.predict_proba(arys)[:,1]
    if y_label == 0:
        conf = model.predict_proba(arys)[:,0]
    return statistics.mean(conf)

In [ ]:
coalition1 = get_coalition()
instance1 = X_test.iloc[1]
print(len(X_test))

In [ ]:
std_devs = []
for i in X_test.columns:
    stdev = statistics.stdev(X_test[i])
    std_devs.append(stdev)
print(std_devs)

In [ ]:
def get_nbr_conf(instance, coalition):
    """
    For every instance perturb neighbor and calculate the coefficient for the neighbor.
    """
    neighbors = []
    for i in range(len(instance) * 3):
        z_neighbor = []
        for count, j in enumerate(instance):
            if coalition[count] == 1:
                z_neighbor.append(j)
            else:
                stdev = std_devs[count]
                perturbed = np.random.normal(j, stdev)
                z_neighbor.append(perturbed)
        neighbors.append(z_neighbor)
    confidences = model.predict_proba(neighbors)[:,1]
    pos_conf, neg_conf = ([] for i in range(2))
    for count, i in enumerate(confidences):
        if i >= .5:
            pos_conf.append(confidences[count])
        else:
            neg_conf.append(confidences[count])
    return pos_conf, neg_conf

In [ ]:
def reward_function(coalition, instance, lambda_g, lambda_p, y_label):
    """
    takes coalitions, selected instances, constant for generility and precision along with label
    returns the calculated reward.
    """
    pos_conf, neg_conf = get_nbr_conf(instance, coalition)
    sum_g = 0
    sum_p = 0
    if y_label == 1:
        for j in pos_conf:
            sum_g += ((lambda_g * j) / len(pos_conf))
        for j in neg_conf:
            sum_p += ((lambda_p * (j - 1)) / len(neg_conf))
    if y_label == 0:
        for j in neg_conf:
            sum_g += ((lambda_g * (1 - j)) / len(neg_conf))
        for j in pos_conf:
            sum_p += ((lambda_p * ((1 - j) - 1)) / len(pos_conf))
    sum_coalition = (sum_g + sum_p)
    return get_mean_constant(coalition, instance, y_label) + sum_coalition

In [ ]:
#Select the constant carefully to adjust precision and generility contribution to the equation.
lambda_g = 1
lambda_p = 1
# reward1 = reward_function(coalition1, instance1, lambda_g, lambda_p)

In [ ]:
# print(reward1)

In [ ]:
import math
def get_kernel(coalition):
    M = len(coalition) - 1
    pres = sum(coalition)
    choose = math.factorial(M) / (math.factorial(pres) * math.factorial(M - pres))
    kernel = (M - 1) / ((choose)*(pres)*(M - pres))
    return kernel

In [ ]:
import random
from sklearn.linear_model import LinearRegression
def get_coefficients_by_index(instance_ind, SHAPLEYS):
    instance = X_test.iloc[instance_ind]
    y_conf = prob1[instance_ind]
    y_label = 0
    if y_conf >= 0.5:
        y_label = 1
    else:
        y_label = 0
    lin_X = []
    lin_y = []
    kernels = []
    for i in range(len(instance) * 3):
        #print(i, end=" ")
        coalition = get_coalition()
        reward = reward_function(coalition, instance, lambda_g, lambda_p, y_label)
        lin_X.append(coalition)
        lin_y.append(reward)
        kernels.append(get_kernel(coalition))
    reg = LinearRegression()
    reg.fit(lin_X, lin_y, kernels)
    cur_dict_shap = {'coef' : reg.coef_, 'org_label' : y_test.iloc[instance_ind], 
            'org_df_index' : X_test.iloc[instance_ind].name, 'relative_index' : instance_ind, 'y_conf' : y_conf}
    SHAPLEYS.put(cur_dict_shap)
    #return 

## Multicore

- For the calculation of perturbed neighbors, every calculatoin can ve very time consuming.
- Try to use maximum amount of available CPU for the calculation
- For our experiment, utilizing 5 CPU requires more than 2.5 hours for the calculation. By utilizing 20 CPU the time is 0.5 hours!

In [ ]:
from multiprocessing import cpu_count

cpu_count()

In [ ]:
import multiprocessing as mp
from multiprocessing import Pool
from tqdm.contrib.concurrent import process_map
from multiprocessing import Manager
from functools import partial

In [ ]:
no_of_cpu_to_use = 5

In [ ]:
results = []
for chunk in tqdm(range(0, len(X_test), no_of_cpu_to_use)):
    SHAPLEYS = mp.Queue()
    processes = [mp.Process(target=get_coefficients_by_index, args=(x, SHAPLEYS)) for x in tqdm(range(chunk, chunk+no_of_cpu_to_use, 1))]
    # Run processes
    for p in tqdm(processes):
        p.start()

    # Exit the completed processes
    for p in processes:
        p.join()
    results += [SHAPLEYS.get() for p in processes]
    print(len(results))

In [ ]:
df_result = pd.DataFrame(results)
df_result.to_pickle('reward_power_p1_g1_v2.pickle')

In [ ]:
df_gaps_reward = df_result.copy()

# Explainability

In [ ]:
np_arr_lime_val_x_test = np.empty((0,X_test.shape[1]), float)
np_arr_lime_imp_x_test = np.empty((0,X_test.shape[1]), float)
np_arr_shap_val_x_test = np.empty((0,X_test.shape[1]), float)
np_arr_shap_imp_x_test = np.empty((0,X_test.shape[1]), float)


explainer_lime = lime.lime_tabular.LimeTabularExplainer(X_train.to_numpy(), feature_names=X_test.columns, class_names=target_names, discretize_continuous=False)
explainer_shap = shap.TreeExplainer(rf, feature_perturbation = "interventional") 


#for _idx in tqdm(range(X_test.shape[0])):
for _idx in tqdm(range(X_test.shape[0])):
    #lime
    
    exp_lime = explainer_lime.explain_instance(X_test.iloc[_idx], rf.predict_proba, num_features = len(X_test.columns))
    #lime_pos_features = [y[0] for indx,y in enumerate(exp_lime.as_map()[1]) if y[1] >= 0]

    #importancy of features [starting from 0]
    lime_importency = np.argsort([y[0] for y in exp_lime.as_map()[1]])
    np_arr_lime_imp_x_test = np.append(np_arr_lime_imp_x_test, [lime_importency], axis=0)
    ###########np_arr_lime_imp_x_test[_idx] = lime_importency
    
    #values for corresponding indexes
    sorted_indexes = sorted(exp_lime.as_map()[1], key=lambda k: k[0]) 
    lime_sorted_value = np.array([y[1] for y in sorted_indexes])
    np_arr_lime_val_x_test = np.append(np_arr_lime_val_x_test, [lime_sorted_value], axis=0)
    #############np_arr_lime_val_x_test[_idx] = lime_sorted_value
    
    #shap
    
    shap_values_individual = explainer_shap.shap_values(X_test.iloc[_idx, :])
    shap_features = list(np.argsort(shap_values_individual[1])[::-1])
    
    
    np_arr_shap_val_x_test = np.append(np_arr_shap_val_x_test,[shap_values_individual[1]], axis=0)
    np_arr_shap_imp_x_test = np.append(np_arr_shap_imp_x_test, [np.argsort(shap_values_individual[1])[::-1]], axis=0)
    


In [ ]:
y_test_predicted = rf.predict(X_test)
np_arr_lime_imp_x_test_correctly_classified_attack = np_arr_lime_imp_x_test[ (y_test == y_test_predicted) & (y_test_predicted == 1)]
np_arr_lime_imp_x_test_correctly_classified_natural = np_arr_lime_imp_x_test[ (y_test == y_test_predicted) & (y_test_predicted == 0)]

X_test_correctly_classified_attack = X_test.values[(y_test == y_test_predicted) & (y_test_predicted == 1)]
X_test_correctly_classified_natural = X_test.values[(y_test == y_test_predicted) & (y_test_predicted == 0)]

feature_attack_sorted = np_arr_lime_imp_x_test_correctly_classified_attack.copy()
feature_natural_sorted = np_arr_lime_imp_x_test_correctly_classified_natural.copy()

In [ ]:
all_k_attack_list_lime_original = []
for k in tqdm(range(1, 20, 1), desc = "k "):
    features_to_consider = k
    list_attack_top_features_nutral_count_k = []

    for attack_index in range(feature_attack_sorted.shape[0]):
        top_attack_features = feature_attack_sorted[attack_index][:features_to_consider]
        #print('top_attack_features : ', top_attack_features)
        top_attack_value = X_test_correctly_classified_attack[attack_index, top_attack_features.astype(int)]

        total_natural_count = 0
        top_features_matched_natural_count = 0

        for natural_index in range(feature_natural_sorted.shape[0]):

            #current_natural_features = np.where(x_test_correctly_classified_natural[natural_index] == 1)[1]
            #current_natural_features = feature_natural_sorted[natural_index, :features_to_consider]
            current_natural_value = X_test_correctly_classified_natural[natural_index, top_attack_features.astype(int)]
            

            #checking top features
            #top_feature_match_count = len(np.intersect1d(top_attack_features, current_natural_features)) 
            '''
            top_feature_match_count = 0
            for current_top_feature in top_malware_features:
                if current_top_feature in current_goodware_features:
                    top_feature_match_count = top_feature_match_count +1
            '''

            #if top_feature_match_count >= features_to_consider:
            if np.array_equal(top_attack_value, current_natural_value):
                top_features_matched_natural_count += 1

        current_attack_stat = {'index' : attack_index, "natural_matching_top_k_feature" : top_features_matched_natural_count}

        #print(current_malware_stat)
        list_attack_top_features_nutral_count_k.append(current_attack_stat)
    current_k = {'k': k, 'list_attack_top_features_nutral_count_k': list_attack_top_features_nutral_count_k}
    all_k_attack_list_lime_original.append(current_k)

In [ ]:
lst_dic_match_value_lime_original = []

for i_attack in all_k_attack_list_lime_original: 
    dic_match_value = {}
    dic_match_value["k"] = i_attack["k"]
    dic_match_value["lst_number_of_matched"] = [spec_val["natural_matching_top_k_feature"] for spec_val in i_attack["list_attack_top_features_nutral_count_k"]]
    lst_dic_match_value_lime_original.append(dic_match_value)

In [ ]:
lst_sum_lime_original = []
for _item in lst_dic_match_value_lime_original:
    _tmp_sum = 0
    for _i_item in _item['lst_number_of_matched']:
        if _i_item > 0:
            _tmp_sum+= 1
    lst_sum_lime_original.append(_tmp_sum)

## Shap

In [ ]:
y_test_predicted = rf.predict(X_test)
np_arr_shap_imp_x_test_correctly_classified_attack = np_arr_shap_imp_x_test[ (y_test == y_test_predicted) & (y_test_predicted == 1)]
np_arr_shap_imp_x_test_correctly_classified_natural = np_arr_shap_imp_x_test[ (y_test == y_test_predicted) & (y_test_predicted == 0)]

In [ ]:
X_test_correctly_classified_attack = X_test.values[(y_test == y_test_predicted) & (y_test_predicted == 1)]
X_test_correctly_classified_natural = X_test.values[(y_test == y_test_predicted) & (y_test_predicted == 0)]

In [ ]:
feature_attack_sorted = np_arr_shap_imp_x_test_correctly_classified_attack.copy()
feature_natural_sorted = np_arr_shap_imp_x_test_correctly_classified_natural.copy()

In [ ]:
all_k_attack_list_shap_original = []
for k in tqdm(range(1, 20, 1), desc = "k "):
    features_to_consider = k
    list_attack_top_features_nutral_count_k = []

    for attack_index in range(feature_attack_sorted.shape[0]):
        top_attack_features = feature_attack_sorted[attack_index][:features_to_consider]
        #print('top_attack_features : ', top_attack_features)
        top_attack_value = X_test_correctly_classified_attack[attack_index, top_attack_features.astype(int)]

        total_natural_count = 0
        top_features_matched_natural_count = 0

        for natural_index in range(feature_natural_sorted.shape[0]):

            #current_natural_features = np.where(x_test_correctly_classified_natural[natural_index] == 1)[1]
            #current_natural_features = feature_natural_sorted[natural_index, :features_to_consider]
            current_natural_value = X_test_correctly_classified_natural[natural_index, top_attack_features.astype(int)]
            

            #checking top features
            #top_feature_match_count = len(np.intersect1d(top_attack_features, current_natural_features)) 
            '''
            top_feature_match_count = 0
            for current_top_feature in top_malware_features:
                if current_top_feature in current_goodware_features:
                    top_feature_match_count = top_feature_match_count +1
            '''

            #if top_feature_match_count >= features_to_consider:
            if np.array_equal(top_attack_value, current_natural_value):
                top_features_matched_natural_count += 1

        current_attack_stat = {'index' : attack_index, "natural_matching_top_k_feature" : top_features_matched_natural_count}

        #print(current_malware_stat)
        list_attack_top_features_nutral_count_k.append(current_attack_stat)
    current_k = {'k': k, 'list_attack_top_features_nutral_count_k': list_attack_top_features_nutral_count_k}
    all_k_attack_list_shap_original.append(current_k)

In [ ]:
lst_dic_match_value_shap_original = []

for i_attack in all_k_attack_list_shap_original: 
    dic_match_value = {}
    dic_match_value["k"] = i_attack["k"]
    dic_match_value["lst_number_of_matched"] = [spec_val["natural_matching_top_k_feature"] for spec_val in i_attack["list_attack_top_features_nutral_count_k"]]
    lst_dic_match_value_shap_original.append(dic_match_value)

In [ ]:
lst_sum_shap_original = []
for _item in lst_dic_match_value_shap_original:
    _tmp_sum = 0
    for _i_item in _item['lst_number_of_matched']:
        if _i_item > 0:
            _tmp_sum+= 1
    lst_sum_shap_original.append(_tmp_sum)

In [ ]:
coun_shap_sum = np.array([_x["lst_number_of_matched"] for _x in lst_dic_match_value_shap_original[:10]])

## GAPS

In [ ]:
def get_featureimportance_by_value(cur_row_vals):
    return [np.argsort(cur_row_vals)[::-1]]


df_gaps_reward['feature_imp'] = df_gaps_reward.apply(lambda x: get_featureimportance_by_value(x['coef']), axis=1)

In [ ]:
np_arr_gaps_imp_x_test_correctly_classified_attack_list = df_gaps_reward['feature_imp'].values[ (y_test == y_test_predicted) & (y_test_predicted == 1)]
np_arr_gaps_imp_x_test_correctly_classified_natural_list = df_gaps_reward['feature_imp'].values[ (y_test == y_test_predicted) & (y_test_predicted == 0)]

np_arr_gaps_imp_x_test_correctly_classified_attack = np.array([x[0] for x in np_arr_gaps_imp_x_test_correctly_classified_attack_list])
np_arr_gaps_imp_x_test_correctly_classified_natural = np.array([x[0] for x in np_arr_gaps_imp_x_test_correctly_classified_natural_list])

feature_attack_sorted = np_arr_gaps_imp_x_test_correctly_classified_attack.copy()
feature_natural_sorted = np_arr_gaps_imp_x_test_correctly_classified_natural.copy()

In [ ]:
all_k_attack_list_gaps_original = []
for k in tqdm(range(1, 20, 1), desc = "k "):
    features_to_consider = k
    list_attack_top_features_nutral_count_k = []

    for attack_index in range(feature_attack_sorted.shape[0]) :
        top_attack_features = feature_attack_sorted[attack_index][:features_to_consider]
        #print('top_attack_features : ', top_attack_features)
        top_attack_value = X_test_correctly_classified_attack[attack_index, top_attack_features.astype(int)]

        total_natural_count = 0
        top_features_matched_natural_count = 0

        for natural_index in range(feature_natural_sorted.shape[0]):

            #current_natural_features = np.where(x_test_correctly_classified_natural[natural_index] == 1)[1]
            #current_natural_features = feature_natural_sorted[natural_index, :features_to_consider]
            current_natural_value = X_test_correctly_classified_natural[natural_index, top_attack_features.astype(int)]
            

            #checking top features
            #top_feature_match_count = len(np.intersect1d(top_attack_features, current_natural_features)) 
            '''
            top_feature_match_count = 0
            for current_top_feature in top_malware_features:
                if current_top_feature in current_goodware_features:
                    top_feature_match_count = top_feature_match_count +1
            '''

            #if top_feature_match_count >= features_to_consider:
            if np.array_equal(top_attack_value, current_natural_value):
                top_features_matched_natural_count += 1

        current_attack_stat = {'index' : attack_index, "natural_matching_top_k_feature" : top_features_matched_natural_count}

        #print(current_malware_stat)
        list_attack_top_features_nutral_count_k.append(current_attack_stat)
    current_k = {'k': k, 'list_attack_top_features_nutral_count_k': list_attack_top_features_nutral_count_k}
    all_k_attack_list_gaps_original.append(current_k)

In [ ]:
lst_dic_match_value_gaps_original = []

for i_attack in all_k_attack_list_gaps_original: 
    dic_match_value = {}
    dic_match_value["k"] = i_attack["k"]
    dic_match_value["lst_number_of_matched"] = [spec_val["natural_matching_top_k_feature"] for spec_val in i_attack["list_attack_top_features_nutral_count_k"]]
    lst_dic_match_value_gaps_original.append(dic_match_value)

In [ ]:
lst_sum_gaps_original = []
for _item in lst_dic_match_value_gaps_original:
    _tmp_sum = 0
    for _i_item in _item['lst_number_of_matched']:
        if _i_item > 0:
            _tmp_sum+= 1
    lst_sum_gaps_original.append(_tmp_sum)

In [ ]:
coun_gaps_sum = np.array([_x["lst_number_of_matched"] for _x in lst_dic_match_value_gaps_original[:21]])

## Compare and analysis

### Plot

In [ ]:
lst_sum_lime_original_cent = [(_x/feature_attack_sorted.shape[0])*100 for _x in lst_sum_lime_original]
lst_sum_shap_original_cent = [(_x/feature_attack_sorted.shape[0])*100 for _x in lst_sum_shap_original]
lst_sum_shap_our_cent = [(_x/feature_attack_sorted.shape[0])*100 for _x in lst_sum_gaps_original]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

with sns.axes_style("whitegrid"):
    data = [lst_sum_lime_original_cent[:20],
    lst_sum_shap_original_cent[:20],
    lst_sum_shap_our_cent[:20]]
    X = np.arange(1,20,1)
    fig = plt.figure(figsize=(20, 5))
    ax = fig.add_axes([0,1,1,1])
    b1= ax.bar(X + 0.00, data[0], width = 0.25, color = 'w', hatch = '//', edgecolor='k')
    b2 = ax.bar(X + 0.25, data[1], width = 0.25, color = 'w', hatch = '..', edgecolor='k')
    b3 = ax.bar(X + 0.50, data[2], width = 0.25, color = 'w', hatch = '++', edgecolor='k')
    
    #plt.xticks(np.arange(0,11,1))
    #plt.xticks(ha='right')
#     plt.xticks(X + 0.25, ('1', '2', '3', '4', '5', '6', '7', '8', '9', '10'))
    plt.xticks(X + 0.25, ('1', '2', '3', '4', '5', '6', '7', '8', '9', '10',
                         '11', '12', '13', '14', '15', '16', '17', '18', '19'))

    plt.yticks(np.arange(0,40,5))
    
    plt.legend([b1,b2,b3],["Lime", "Shap", "GAPS"], bbox_to_anchor=(0.0, 0.35, 1.0, 1.0), loc='right',
           ncol=1,  borderaxespad=2.0, prop={'size': 14})
    
    plt.xlabel('Number of K considered')
    plt.ylabel('% of natural event matches with attack')
    #plt.title('Here goes title of the plot')
    plt.show()

In [ ]:
coun_gaps_sum = np.array([_x["lst_number_of_matched"] for _x in lst_dic_match_value_gaps_original[:11]])

lst_sum_lime_original_cent = [(_x/feature_attack_sorted.shape[0])*100 for _x in lst_sum_lime_original]
lst_sum_shap_original_cent = [(_x/feature_attack_sorted.shape[0])*100 for _x in lst_sum_shap_original]
lst_sum_shap_our_cent = [(_x/feature_attack_sorted.shape[0])*100 for _x in lst_sum_gaps_original]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

with sns.axes_style("whitegrid"):
    data = [lst_sum_lime_original_cent[:10],
    lst_sum_shap_original_cent[:10],
    lst_sum_shap_our_cent[:10]]
    X = np.arange(1,11,1)
    fig = plt.figure(figsize=(10, 5))
    ax = fig.add_axes([0,1,1,1])
    b1= ax.bar(X + 0.00, data[0], width = 0.25, color = 'w', hatch = '///', edgecolor='k')
    b2 = ax.bar(X + 0.25, data[1], width = 0.25, color = 'w', hatch = '...', edgecolor='k')
    b3 = ax.bar(X + 0.50, data[2], width = 0.25, color = 'w', hatch = '+++', edgecolor='k')
    
    #plt.xticks(np.arange(0,11,1))
    #plt.xticks(ha='right')
    plt.xticks(X + 0.25, ('1', '2', '3', '4', '5', '6', '7', '8', '9', '10'))

    plt.yticks(np.arange(0,110,10))
    
    plt.legend([b1,b2,b3],["Lime", "Shap", "GAPS"], bbox_to_anchor=(0.0, 0.35, 1.0, 1.0), loc='right',
           ncol=1,  borderaxespad=2.0, prop={'size': 14})
    
    plt.xlabel('Number of K considered')
    plt.ylabel('% of natural event matches with attack')
    #plt.title('Here goes title of the plot')
    plt.show()

### Generality

#### Distance Measure

In [ ]:
from math import*

def cal_jaccard_distance(x, y):
    intersection_cardinality = len(set.intersection(*[set(x), set(y)]))
    union_cardinality = len(set.union(*[set(x), set(y)]))
    return intersection_cardinality/float(union_cardinality)

def cal_intersection_size(x, y):
    return len(set.intersection(*[set(x), set(y)]))
    
def euclidean_distance(x,y):
    return sqrt(sum(pow(a-b,2) for a, b in zip(x, y)))


def square_rooted(x): 
    return round(sqrt(sum([a*a for a in x])),3)
 
def cosine_similarity(x,y): 
    numerator = sum(a*b for a,b in zip(x,y))
    denominator = square_rooted(x)*square_rooted(y)
    return round(numerator/float(denominator),3)

In [ ]:
def euclidian_distance(a, b):
    return np.sqrt(np.sum((a-b)**2, axis=1))

In [ ]:
no_of_feature_considered = 50

In [ ]:
jaccard_vec = np.array([0.0 for x in range(no_of_feature_considered)])
jaccard_vec_cal = np.array([0.0 for x in range(no_of_feature_considered)])
intersection_dis_vec = np.array([0.0 for x in range(no_of_feature_considered)])
cosine_dist_vec = np.array([0.0 for x in range(no_of_feature_considered)])
euclidean_vec = np.array([0.0 for x in range(no_of_feature_considered)])

In [ ]:
from sklearn.metrics import jaccard_score

for _idx in tqdm(range(X_test.shape[0])):
    #lime
    
    exp_lime = explainer_lime.explain_instance(X_test.iloc[_idx], rf.predict_proba, num_features = len(X_test.columns))
    lime_pos_features = [y[0] for indx,y in enumerate(exp_lime.as_map()[1]) if y[1] >= 0]
    
    #shap
    shap_values_individual = explainer_shap.shap_values(X_test.iloc[_idx, :])
    shap_features = list(np.argsort(shap_values_individual[1])[::-1])
    
    
    for kth_feature in range(1,no_of_feature_considered):
        lime_k_feature = lime_pos_features[:kth_feature]
        shap_k_feature = shap_features[:kth_feature]
        
        vec_lime_k_feature = np.zeros(len(X_test.columns))
        vec_lime_k_feature[lime_k_feature] = 1
        vec_shap_k_feature = np.zeros(len(X_test.columns))
        vec_shap_k_feature[shap_k_feature] = 1

        jaccard_vec[kth_feature] += float(jaccard_score(vec_lime_k_feature, vec_shap_k_feature))
        
        jaccard_vec_cal[kth_feature] += cal_jaccard_distance(lime_k_feature, shap_k_feature)
        
        intersection_dis_vec[kth_feature] += cal_intersection_size(lime_k_feature, shap_k_feature)
        
        cosine_dist_vec[kth_feature] += cosine_similarity(lime_k_feature, shap_k_feature)
        
        euclidean_vec[kth_feature] += euclidean_distance(lime_k_feature, shap_k_feature)

In [ ]:
jaccard_vec_avg = np.array([0.0 for x in range(no_of_feature_considered)])
jaccard_vec_cal_avg = np.array([0.0 for x in range(no_of_feature_considered)])
intersection_dis_vec_avg = np.array([0.0 for x in range(no_of_feature_considered)])
cosine_dist_vec_avg = np.array([0.0 for x in range(no_of_feature_considered)])
euclidean_vec_avg = np.array([0.0 for x in range(no_of_feature_considered)])


for kth_feature in tqdm(range(no_of_feature_considered)):
    jaccard_vec_avg[kth_feature]=jaccard_vec[kth_feature]/X_test.shape[0]
    jaccard_vec_cal_avg[kth_feature] = jaccard_vec_cal[kth_feature]/X_test.shape[0]
    intersection_dis_vec_avg[kth_feature] = intersection_dis_vec[kth_feature]/X_test.shape[0]
    cosine_dist_vec_avg[kth_feature] = cosine_dist_vec[kth_feature]/X_test.shape[0]
    euclidean_vec_avg[kth_feature] = euclidean_vec[kth_feature]/X_test.shape[0] 

In [ ]:
#plt.figure()
with sns.axes_style("whitegrid"):
    fig = plt.figure(figsize=(15,7))
    #fig.suptitle('Consistancy', fontsize=20)
    ax = plt.axes()
    #plt.plot(np.arange(1,num_feature_considered), jaccard_vec_avg[1:], '^', color='#006aff', label = "Jaccared scikitlearn", linestyle=':')        # specify color by name
    #plt.plot(np.arange(1,num_feature_considered), jaccard_vec_cal_avg[1:], 'v', color='#0b7300', label = "Jaccared calculated", linestyle='-.')           # short color code (rgbcmyk)
    #plt.plot(np.arange(1,no_of_feature_considered), intersection_dis_vec_avg[1:], color='k', label = "Intersection length", marker='8')        # Grayscale between 0 and 1
    #plt.plot(np.arange(1,num_feature_considered), cosine_dist_vec_avg[1:], color='#9500ff', label = "Cosine similarity")     # Hex code (RRGGBB from 00 to FF)
    #plt.plot(np.arange(1,num_feature_considered), euclidean_vec_avg[1:], color=(1.0,0.2,0.3), label = "Euclidian similarity") # RGB tuple, values 0 to 1

    plt.plot(np.arange(1,no_of_feature_considered), intersection_dis_vec_avg[1:], color='k', label = "Intersection length", marker='8')        # Grayscale between 0 and 1
    #plt.plot(np.arange(1,no_of_feature_considered), euclidean_vec_avg[1:], '^', color='k', label = "Jaccared scikitlearn", linestyle=':')        # specify color by name
    
    plt.legend(numpoints=1)
    plt.xticks(np.arange(0,55,5))
    #plt.yticks(np.arange(0,300,50))
    plt.yticks(np.arange(0,30,5))
    plt.xlabel('Number of K considered', fontsize=18)
    plt.ylabel('% of intersection points', fontsize=18)
    plt.show()


### Calculation Generality

In [ ]:
import numpy as np
from scipy.spatial.distance import cdist
from scipy.spatial import KDTree
from scipy.spatial import cKDTree

In [ ]:
data = X_test_correctly_classified_attack.copy()
tree = cKDTree(data)
dists = tree.query(data, 2)
nn_dist = dists[0][:, 1]

In [ ]:
def kneighbors(np_ary_to_find_dist, return_distance=False):
       
        n_neighbors = 375
        dist = []
        neigh_ind = []
        
        point_dist = [euclidian_distance(cur_row, np_ary_to_find_dist) for cur_row in np_ary_to_find_dist]

        for row in point_dist:
            enum_neigh = enumerate(row)
            sorted_neigh = sorted(enum_neigh, key=lambda x: x[1])[1:n_neighbors+1]
    
            ind_list = [tup[0] for tup in sorted_neigh]
            dist_list = [tup[1] for tup in sorted_neigh]
    
            dist.append(dist_list)
            neigh_ind.append(ind_list)
        
        if return_distance:
            return np.array(dist), np.array(neigh_ind)
        
        return np.array(neigh_ind)

In [ ]:
neighbors = kneighbors(X_test_correctly_classified_attack)

jaccard_ve_generalityc = np.array([0.0 for x in range(no_of_feature_considered)])
jaccard_vec_cal_generality = np.array([0.0 for x in range(no_of_feature_considered)])
intersection_dis_vec_generality = np.array([0.0 for x in range(no_of_feature_considered)])
cosine_dist_vec_generality = np.array([0.0 for x in range(no_of_feature_considered)])
euclidean_vec_generality = np.array([0.0 for x in range(no_of_feature_considered)])

In [ ]:
no_of_neighbors_to_consider_list = range(1,11,1)#[1, 5, 10]
no_of_attributes_to_consider_list = range(1,11,1)#[3,5,2]

In [ ]:
list_generality_intersection = []

for cur_number_of_neighbors in tqdm(no_of_neighbors_to_consider_list, desc = 'neighbor'):
    for cur_number_of_attributes in tqdm(no_of_attributes_to_consider_list, desc = 'attributes'):
        
        
        
        #list_all_instance_lime_intersection_list = []
        #list_all_instance_shap_intersection_list = []
        
        for cur_index_attack in (range(X_test_correctly_classified_attack.shape[0])):#X_test_correctly_classified_attack.shape[0])):
            #select k nearest neighbors
            dic_generality_intersection = {}
            
            cur_selected_neighbors_indxs = neighbors[cur_index_attack]
            cur_selected_k_neighbors_indxs = cur_selected_neighbors_indxs[:cur_number_of_neighbors]
            #print('cur_selected_k_neighbors_indxs', ' >> ', cur_selected_k_neighbors_indxs)

            lime_imp_cur_attack_all_attrib = np_arr_lime_imp_x_test_correctly_classified_attack[cur_index_attack]
            lime_imp_cur_attack_n_attrib = lime_imp_cur_attack_all_attrib[:cur_number_of_attributes]
            #print('lime_imp_cur_attack_n_attrib', '>> ', lime_imp_cur_attack_n_attrib)

            shap_imp_cur_attack_all_attrib = np_arr_shap_imp_x_test_correctly_classified_attack[cur_index_attack]
            shap_imp_cur_attack_n_attrib = shap_imp_cur_attack_all_attrib[:cur_number_of_attributes]
            
            gaps_imp_cur_attack_all_attrib = np_arr_gaps_imp_x_test_correctly_classified_attack[cur_index_attack]
            gaps_imp_cur_attack_n_attrib = gaps_imp_cur_attack_all_attrib[:cur_number_of_attributes]
            

            list_lime_intersection_sizes = []
            list_shap_intersection_sizes = []
            list_gaps_intersection_sizes = []
            for cur_neighbor_instance_indx in cur_selected_k_neighbors_indxs:

                #lime
                lime_imp_cur_neighbor_all_attribs = np_arr_lime_imp_x_test_correctly_classified_attack[cur_neighbor_instance_indx]
                lime_imp_cur_neighbor_n_attribs = lime_imp_cur_neighbor_all_attribs[:cur_number_of_attributes]
                #print('lime_imp_cur_neighbor_n_attribs', '>> ', lime_imp_cur_neighbor_n_attribs)
                itersection_size_lime = cal_intersection_size(lime_imp_cur_attack_n_attrib, lime_imp_cur_neighbor_n_attribs)
                list_lime_intersection_sizes.append(itersection_size_lime)


                #shap
                shap_imp_cur_neighbor_all_attribs = np_arr_shap_imp_x_test_correctly_classified_attack[cur_neighbor_instance_indx]
                shap_imp_cur_neighbor_n_attribs = shap_imp_cur_neighbor_all_attribs[:cur_number_of_attributes]
                #print('shap_imp_cur_neighbor_n_attribs', '>> ', shap_imp_cur_neighbor_n_attribs)
                itersection_size_shap = cal_intersection_size(shap_imp_cur_attack_n_attrib, shap_imp_cur_neighbor_n_attribs)
                list_shap_intersection_sizes.append(itersection_size_shap) 
                
                #gaps
                gaps_imp_cur_neighbor_all_attribs = np_arr_gaps_imp_x_test_correctly_classified_attack[cur_neighbor_instance_indx]
                gaps_imp_cur_neighbor_n_attribs = gaps_imp_cur_neighbor_all_attribs[:cur_number_of_attributes]
                
                #print('shap_imp_cur_neighbor_n_attribs', '>> ', shap_imp_cur_neighbor_n_attribs)
                itersection_size_gaps = cal_intersection_size(gaps_imp_cur_attack_n_attrib, gaps_imp_cur_neighbor_n_attribs)
                list_gaps_intersection_sizes.append(itersection_size_gaps)                 

            #list_all_instance_lime_intersection_list.append(list_lime_intersection_sizes)
            #list_all_instance_shap_intersection_list.append(list_shap_intersection_sizes)
            #print(list_shap_intersection_sizes)
            dic_generality_intersection['no_of_neighbors'] = cur_number_of_neighbors
            dic_generality_intersection['no_of_attributes'] = cur_number_of_attributes
            dic_generality_intersection['lime_intersection_len_list'] = list_lime_intersection_sizes
            dic_generality_intersection['shap_intersection_len_list'] = list_shap_intersection_sizes
            dic_generality_intersection['gaps_intersection_len_list'] = list_gaps_intersection_sizes
    
            list_generality_intersection.append(dic_generality_intersection)

In [ ]:
df_generility_intersection_details = pd.DataFrame(list_generality_intersection)

In [ ]:
df_generility_intersection_details['lime_intersection_len_list_max'] = df_generility_intersection_details['lime_intersection_len_list'].apply(max)
df_generility_intersection_details['lime_intersection_len_list_min'] = df_generility_intersection_details['lime_intersection_len_list'].apply(min)
df_generility_intersection_details['lime_intersection_len_list_mean'] = df_generility_intersection_details['lime_intersection_len_list'].apply(np.mean)

In [ ]:
df_generility_intersection_details['shap_intersection_len_list_max'] = df_generility_intersection_details['shap_intersection_len_list'].apply(max)
df_generility_intersection_details['shap_intersection_len_list_min'] = df_generility_intersection_details['shap_intersection_len_list'].apply(min)
df_generility_intersection_details['shap_intersection_len_list_mean'] = df_generility_intersection_details['shap_intersection_len_list'].apply(np.mean)

In [ ]:
df_generility_intersection_details['gaps_intersection_len_list_max'] = df_generility_intersection_details['gaps_intersection_len_list'].apply(max)
df_generility_intersection_details['gaps_intersection_len_list_min'] = df_generility_intersection_details['gaps_intersection_len_list'].apply(min)
df_generility_intersection_details['gaps_intersection_len_list_mean'] = df_generility_intersection_details['gaps_intersection_len_list'].apply(np.mean)

In [ ]:
list_columns_to_consider = ['no_of_neighbors', 'no_of_attributes',
                            'lime_intersection_len_list_max', 'lime_intersection_len_list_min', 'lime_intersection_len_list_mean',
                           'shap_intersection_len_list_max', 'shap_intersection_len_list_min', 'shap_intersection_len_list_mean',
                           'gaps_intersection_len_list_max', 'gaps_intersection_len_list_min', 'gaps_intersection_len_list_mean']

list_columns_to_group = ['no_of_neighbors', 'no_of_attributes']

In [ ]:
df_generility_intersection_summary = df_generility_intersection_details[list_columns_to_consider].groupby(list_columns_to_group).agg(
    {
        "lime_intersection_len_list_max": [np.mean, np.max, np.min], "shap_intersection_len_list_max": [np.mean, np.max, np.min],"gaps_intersection_len_list_max": [np.mean, np.max, np.min],
        "lime_intersection_len_list_mean": [np.mean, np.max, np.min], "shap_intersection_len_list_mean": [np.mean, np.max, np.min], "gaps_intersection_len_list_mean": [np.mean, np.max, np.min],
        "lime_intersection_len_list_min": [np.mean, np.max, np.min], "shap_intersection_len_list_min": [np.mean, np.max, np.min], "gaps_intersection_len_list_min": [np.mean, np.max, np.min],
    })

In [ ]:
display(HTML(df_generility_intersection_summary.to_html()))

In [ ]:
df_generility_intersection_summary.to_csv('generality_power_g1_p1_v2.csv')